In [8]:
#imports
import numpy as np 
import os
import scipy
from scipy.io import loadmat
import pandas as pd
import tensorflow as tf 
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score, cohen_kappa_score, hamming_loss, zero_one_loss, coverage_error, average_precision_score
from sklearn.model_selection import train_test_split
import random 
import matplotlib.pyplot as plt
from time import strftime, localtime
from sklearn.model_selection import KFold
random.seed = 11

import warnings
warnings.filterwarnings('ignore')


In [9]:
# Reading dataset
educationDataset = loadmat("/home/karim/Documents/research/sourceCode/MLML/mlml_weightedLoss/Datasets/MulanDatasets/delicious.mat")

features = educationDataset['train_data']
test_features = educationDataset['test_data']

labels = educationDataset['train_target'].T
test_labels = educationDataset['test_target'].T

"""
Split ratio is strange: 40% training and 60% testing, we merge and data and resplit with 70/30 split
"""
features = np.append(features,test_features,axis = 0)
labels = np.append(labels,test_labels,axis = 0)

print("Number of samples is: {}".format(len(features)))
print("#features is: {}, #labels is: {}".format(features.shape[1],labels.shape[1]))
print("Ratio of positive labels is: {:.2f}%".format(100 * labels.sum()/(labels.shape[0]*labels.shape[1])))

splitter = KFold(n_splits=5, shuffle=True, random_state=0)

#features, test_features, labels, test_labels = train_test_split(features, labels, test_size=0.33, random_state=0)

Number of samples is: 16105
#features is: 500, #labels is: 983
Ratio of positive labels is: 1.93%


## NN approach

In [10]:
# define helper functions
def get_weights(shape):
    w = tf.Variable(tf.truncated_normal(shape, stddev=0.1))
    #variable_summaries(w)
    return w

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    b = tf.Variable(initial)
    #variable_summaries(b)
    return b

def full_layer(input, size):
    in_size = int(input.get_shape()[1])
    W = get_weights([in_size, size])
    b = bias_variable([size])
    return tf.matmul(input, W) + b

def weighted_loss(y_true, y_pred, positive_weights, negative_weights):
    # clip to prevent NaN's and Inf's
    y_pred = tf.clip_by_value(y_pred, 1e-7, 1-1e-7, name=None)
    #y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    # calc
    loss = (-y_true * tf.log(y_pred) * positive_weights) - ((1.0 - y_true) * tf.log(1.0 - y_pred) * negative_weights)
    loss = tf.reduce_mean(loss)
    return loss

def evaluation_report(folds_metrics,evaluation_file_path,header_note):
    metrics_mean = np.mean(folds_metrics, axis = 0)
    metrics_std = np.std(folds_metrics, axis = 0) * 1.96 # for 95% confidence interval
    print(header_note)
    print("===================")
    print("Test set evaluation")
    print("Hamming loss is:{:.3f} (+/-{:.3f})".format(metrics_mean[0],metrics_std[0]))
    print("Zero-one loss is:{:.3f} (+/-{:.3f})".format(metrics_mean[1],metrics_std[1]))
    print("Coverage error is:{:.3f} (+/-{:.3f})".format(metrics_mean[2],metrics_std[2]))
    print("F1 is:{:.3f} (+/-{:.3f})".format(metrics_mean[3],metrics_std[3]))
    print("Recall is:{:.3f} (+/-{:.3f})".format(metrics_mean[4],metrics_std[4]))
    print("Precision is:{:.3f} (+/-{:.3f})".format(metrics_mean[5],metrics_std[5]))
    print("Average Precision is:{:.3f} (+/-{:.3f})".format(metrics_mean[6],metrics_std[6]))
    
    with open(evaluation_file_path, "a+") as f:
        f.write("===================\n" + header_note + "\n")
        f.write("Hamming loss is:{:.3f} (+/-{:.3f})".format(metrics_mean[0],metrics_std[0]) + "\n"+
                "Zero-one loss is:{:.3f} (+/-{:.3f})".format(metrics_mean[1],metrics_std[1]) + "\n"+
                "Coverage error is:{:.3f} (+/-{:.3f})".format(metrics_mean[2],metrics_std[2]) + "\n"+
                "F1 is:{:.3f} (+/-{:.3f})".format(metrics_mean[3],metrics_std[3]) + "\n"+
                "Recall is:{:.3f} (+/-{:.3f})".format(metrics_mean[4],metrics_std[4]) + "\n"+
                "Precision is:{:.3f} (+/-{:.3f})".format(metrics_mean[5],metrics_std[5]) + "\n"+
                "Average Precision is:{:.3f} (+/-{:.3f})".format(metrics_mean[6],metrics_std[6])
               + "\n\n")
        
def evaluation_metrics_per_fold(test_labels, test_probs):
    # ignoring auc for now because of error of undefined case of a class with no ones
    try:
        auc = roc_auc_score(test_labels, val_output)
    except:
        pass
    HL = hamming_loss(test_labels, np.round(test_probs))
    one_loss = zero_one_loss(test_labels, np.round(test_probs))
    cover = coverage_error(test_labels, test_probs)
    f1 = f1_score(test_labels, np.round(test_probs),average="micro")
    recall = recall_score(test_labels, np.round(test_probs),average="micro")
    precision = precision_score(test_labels, np.round(test_probs),average="micro")
    average_precision = average_precision_score(test_labels, test_probs,average="micro")
    metrics = [HL, one_loss, cover, f1, recall, precision, average_precision]
    return metrics
               

In [11]:
# Define a 3 layers network to train 
input_shape = features.shape[1]
output_shape = labels.shape[1]
hidden_layer_1_shape = 300
hidden_layer_2_shape = 200
hidden_layer_3_shape = 100

y = tf.placeholder(tf.float32, [None, output_shape], name="true_labels")
x_input = tf.placeholder(tf.float32, [None,input_shape],name="input_layer")
current_keep_prob = tf.placeholder(tf.float32, name="dropout_rate")
h1 = tf.nn.tanh(full_layer(x_input, hidden_layer_1_shape))
h2 = tf.nn.tanh(full_layer(h1, hidden_layer_2_shape))
h3 = tf.nn.tanh(full_layer(h2, hidden_layer_3_shape))
dropped = tf.nn.dropout(h3, keep_prob=current_keep_prob)
logits = full_layer(dropped,output_shape)
output = tf.nn.sigmoid(logits)

In [12]:
# Prepare results report 
Experiment_name = "delicious_weightedloss"
experiment_time = strftime("%d-%m_%H:%M", localtime())
saving_dir = "/home/karim/Documents/research/sourceCode/MLML/mlml_weightedLoss/Experiment_results/"
exp_dir = os.path.join(saving_dir,Experiment_name,experiment_time)
os.makedirs(exp_dir)
model_output_dir = os.path.join(exp_dir,"model_output")
os.mkdir(model_output_dir)

## Train on complete dataset

In [13]:
# Define model parameters
# using weighted cross entropy because dataset is sparse and we need to weight positives more
#loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=y))
POSITIVE_WEIGHT = 5
loss = tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(logits=logits, labels=y,pos_weight = POSITIVE_WEIGHT))

# Learning rate decay
global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(learning_rate=0.1, global_step=global_step, decay_steps=1000,
                                          decay_rate=0.95,staircase=True)
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
correct_prediction = tf.equal(tf.round(output), y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [14]:
# training the model
NUM_EPOCHS = 6000

fold = 0
folds_metrics = []
for train_index, test_index in splitter.split(features, labels):
    fold += 1
    print("Train/Evaluate on fold %d" % fold)
    train_features, test_features = features[train_index], features[test_index]
    train_labels, test_labels = labels[train_index], labels[test_index]
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(NUM_EPOCHS):
            epoch_loss, epoch_accuracy,epoch_output, _ = sess.run([loss, accuracy,output, train_step],feed_dict={x_input: 
                                                                                             train_features,y: train_labels,
                                                                                             current_keep_prob: 0.3,})
            if (epoch+1)% 3000 == 0:
                val_losses, val_accuracies, val_output,current_learning_rate = sess.run([loss, accuracy,output,learning_rate],feed_dict={
                                                                                              x_input: test_features,
                                                                                              y:test_labels,
                                                                                              current_keep_prob: 1.0})
                print("Epoch #{}".format(epoch+1), "LR: {:.4f}".format(current_learning_rate),
                      "Loss: {:.4f}".format(epoch_loss), 
                      "accuracy: {:.4f}".format(epoch_accuracy),
                      "Test loss: {:.4f}".format(val_losses), 
                      "Test accuracy: {:.4f}".format(val_accuracies))
    np.savetxt(os.path.join(model_output_dir,'groundtruth_' + str(fold) + '.out'), test_labels, delimiter=',')
    np.savetxt(os.path.join(model_output_dir, 'output' + str(fold) + '.out'), val_output, delimiter=',')
    folds_metrics.append(evaluation_metrics_per_fold(test_labels,val_output))

Train/Evaluate on fold 1


KeyboardInterrupt: 

In [ ]:
# Adjusting threshold 
"""
thresholds = np.arange(0, 1, 0.01)
f1_array = np.zeros((output_shape, len(thresholds)))
for idx in range(output_shape):
    f1_array[idx, :] = [
        f1_score(train_labels[:, idx], np.clip(np.round(epoch_output[:, idx] - threshold + 0.5), 0, 1))
        for threshold in thresholds]
threshold_arg = np.argmax(f1_array, axis=1)
threshold_per_class = thresholds[threshold_arg]

# Applying thresholds optimized per class
model_output_rounded = np.zeros_like(epoch_output)
for idx in range(output_shape):
    model_output_rounded[:, idx] = np.clip(np.round(epoch_output[:, idx] - threshold_per_class[idx] + 0.5), 0, 1)
"""

In [ ]:
#Evaluation
evaluation_report(folds_metrics,os.path.join(exp_dir,"evaluation_report.txt"),"No missing labels")

## With missing labels

In [ ]:
# Create missing labels
ones_indices = np.nonzero(train_labels)
ratio_of_hidden_samples = 0.4
number_of_hidden_samples = int(len(ones_indices[0]) * ratio_of_hidden_samples)
random_indices = random.sample(list(np.arange(len(ones_indices[0]))),number_of_hidden_samples)
indices_to_hide = (ones_indices[0][random_indices] , ones_indices[1][random_indices])
labels_with_missing_positives = np.copy(train_labels)
for counter in range (number_of_hidden_samples):
    labels_with_missing_positives[indices_to_hide[0][counter]][indices_to_hide[1][counter]] = 0

In [ ]:
# Training with missing labels with 40%
NUM_EPOCHS = 15000
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(NUM_EPOCHS):
        epoch_loss, epoch_accuracy,epoch_output, _ = sess.run([loss, accuracy,output, train_step],feed_dict={x_input: 
                                                                                         train_features,y: labels_with_missing_positives,
                                                                                         current_keep_prob: 0.3})
        if (epoch+1)% 2000 == 0:
            val_losses, val_accuracies, val_output,current_learning_rate = sess.run([loss, accuracy,output,learning_rate],feed_dict={
                                                                                          x_input: test_features,
                                                                                          y:test_labels,
                                                                                          current_keep_prob: 1.0})
            print("Epoch #{}".format(epoch+1),  "LR: {:.4f}".format(current_learning_rate),
                  "Loss: {:.4f}".format(epoch_loss), 
                  "accuracy: {:.4f}".format(epoch_accuracy), 
                  "Test loss: {:.4f}".format(val_losses), 
                  "Test accuracy: {:.4f}".format(val_accuracies))
            if (epoch+1) == NUM_EPOCHS:
                print("====================================== \
                      \n\nFinal test accuracy: {:.4f}".format(val_accuracies))


In [ ]:
#Evaluation
evaluation_report(train_labels,epoch_output,test_labels,val_output,
                  os.path.join(exp_dir,"evaluation_report.txt"),"40% missing labels, no weighted loss")

## With fixed negative weights

In [ ]:
# Defining weights for missing labels
train_negative_weights = np.zeros_like(train_labels) + 1 
train_positive_weights = np.zeros_like(train_labels) + 5 # We make positive weight 5 becuase of data imbalance
for counter in range (number_of_hidden_samples):
    train_negative_weights[indices_to_hide[0][counter]][indices_to_hide[1][counter]] = 0

In [ ]:
# Adding the weighted loss to the model
positive_weights = tf.placeholder(tf.float32, [None,output_shape], name = "Positive_weights")
negative_weights = tf.placeholder(tf.float32, [None, output_shape], name="negative_weights")
my_weights_loss = weighted_loss(y_true= y, y_pred= output,
                              positive_weights= positive_weights, negative_weights= negative_weights)

train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(my_weights_loss,global_step=global_step)

In [ ]:
# Training with negative weights!
NUM_EPOCHS = 15000
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(NUM_EPOCHS):
        epoch_my_weights_loss, epoch_loss, epoch_accuracy,epoch_output, _ = sess.run([my_weights_loss, loss, accuracy,output, train_step],feed_dict={x_input: 
                                                                                         train_features,y: labels_with_missing_positives,positive_weights: train_positive_weights,
                                                                                                  negative_weights: train_negative_weights,
                                                                                                  current_keep_prob: 0.3})
        if (epoch+1)% 2000 == 0:
            val_losses, val_accuracies, val_output,current_learning_rate = sess.run([loss, accuracy,output,learning_rate],feed_dict={
                                                                                          x_input: test_features,
                                                                                          y:test_labels,
                                                                                          current_keep_prob: 1.0})
            print("Epoch #{}".format(epoch+1),  "LR: {:.4f}".format(current_learning_rate),
                  "Loss: {:.4f}".format(epoch_loss), 
                  "Weighted Loss: {:.4f}".format(epoch_my_weights_loss),"accuracy: {:.4f}".format(epoch_accuracy), 
                  "Test loss: {:.4f}".format(val_losses), "Test accuracy: {:.4f}".format(val_accuracies))
            if (epoch+1) == NUM_EPOCHS:
                print("====================================== \
                      \n\nFinal test accuracy: {:.4f}".format(val_accuracies))

In [ ]:
#Evaluation
evaluation_report(train_labels,epoch_output,test_labels,val_output,
                  os.path.join(exp_dir,"evaluation_report.txt"),"Weighted loss with 40% missing")